In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
from datetime import timedelta
import gc

In [2]:
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [3]:
df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_2012to2019.pkl").sort_values("Game_ID").reset_index(drop=True)

In [4]:
df.WL = df.WL.replace({"W":1,"L":0})

In [5]:
teams

{'1610612737': 'Atlanta Hawks',
 '1610612738': 'Boston Celtics',
 '1610612739': 'Cleveland Cavaliers',
 '1610612740': 'New Orleans Pelicans',
 '1610612741': 'Chicago Bulls',
 '1610612742': 'Dallas Mavericks',
 '1610612743': 'Denver Nuggets',
 '1610612744': 'Golden State Warriors',
 '1610612745': 'Houston Rockets',
 '1610612746': 'Los Angeles Clippers',
 '1610612747': 'Los Angeles Lakers',
 '1610612748': 'Miami Heat',
 '1610612749': 'Milwaukee Bucks',
 '1610612750': 'Minnesota Timberwolves',
 '1610612751': 'Brooklyn Nets',
 '1610612752': 'New York Knicks',
 '1610612753': 'Orlando Magic',
 '1610612754': 'Indiana Pacers',
 '1610612755': 'Philadelphia 76ers',
 '1610612756': 'Phoenix Suns',
 '1610612757': 'Portland Trail Blazers',
 '1610612758': 'Sacramento Kings',
 '1610612759': 'San Antonio Spurs',
 '1610612760': 'Oklahoma City Thunder',
 '1610612761': 'Toronto Raptors',
 '1610612762': 'Utah Jazz',
 '1610612763': 'Memphis Grizzlies',
 '1610612764': 'Washington Wizards',
 '1610612765': 'De

In [6]:
lastNgames = 10 

In [7]:
# Raw Data
df_ = pd.DataFrame()
for key in teams.keys():
    df_i = df[df.Team_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for time in range(1,lastNgames+1):
        df_team = pd.concat([df_team,pd.DataFrame(df_i.shift(time).loc[:,"FGM":"PTS"].values,
                                                  columns=["LAST_"+str(time)+"_GAME_"+col for col in df_i.loc[:,"FGM":"PTS"].columns]),
                             pd.DataFrame(df_i.shift(time).loc[:,"WL":"WL"].values,
                                          columns=["LAST_"+str(time)+"_GAME_WL"])],axis=1)
    df_team = pd.concat([df_i.loc[:,"Team_ID":"MATCHUP"],df_team],axis=1)
    df_ = pd.concat([df_, df_team],axis=0).reset_index(drop=True)

In [8]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_RAW_2012to2019.pkl")

# Team Data Combine

In [9]:
teams = json.loads(open("/Users/chienan/Pycon/github/SportLottery/nbastat/teamId_mpt").read())

In [10]:
# Raw Data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMGAMELOG_LAST_10_GAEMS_RAW_2012to2019.pkl")

In [11]:
plus_minus = pd.concat([pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl"),
                        pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")],
                       axis=0).reset_index(drop=True)

In [12]:
plus_minus = plus_minus[['GAME_ID', 'TEAM_ID', 'GAME_DATE', 'PLUS_MINUS']]

In [13]:
plus_minus.head()

,GAME_ID,TEAM_ID,GAME_DATE,PLUS_MINUS
0,0021100001,1610612738,2011-12-25,-2.0
1,0021100002,1610612748,2011-12-25,11.0
2,0021100003,1610612741,2011-12-25,1.0
3,0021100004,1610612753,2011-12-25,-8.0
4,0021100005,1610612746,2011-12-25,19.0


In [14]:
lastNgames = 10 

In [15]:
# Raw Data
plus_minus_ = pd.DataFrame()
for key in teams.keys():
    df_i = plus_minus[plus_minus.TEAM_ID==int(key)].reset_index(drop=True)
    df_team = pd.DataFrame()
    for time in range(1,lastNgames+1):
    
        df_team = pd.concat([df_team,pd.DataFrame(df_i.shift(time).loc[:,"PLUS_MINUS":"PLUS_MINUS"].values,
                                                  columns=["LAST_"+str(time)+"_GAME_PLUS_MINUS"])],axis=1)
        
    df_team = pd.concat([df_i.loc[:,"GAME_ID":"TEAM_ID"],df_team],axis=1)
    plus_minus_ = pd.concat([plus_minus_, df_team],axis=0).reset_index(drop=True)

In [16]:
# Home Team Data
nba_home_df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")
nba_home_df = nba_home_df[['TEAM_ID','GAME_ID','PLUS_MINUS']]

# Away Team Data
nba_away_df = pd.read_pickle('/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl')
nba_away_df = nba_away_df[['TEAM_ID','GAME_ID']]

# Join Two Table
nba_df = nba_home_df.merge(nba_away_df, on='GAME_ID', suffixes=['_HOME','_AWAY'],how='inner')
nba_df = nba_df[['GAME_ID','TEAM_ID_HOME','TEAM_ID_AWAY','PLUS_MINUS']]
nba_df.head()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS
0,0021100001,1610612752,1610612738,2.0
1,0021100002,1610612742,1610612748,-11.0
2,0021100003,1610612747,1610612741,-1.0
3,0021100004,1610612760,1610612753,8.0
4,0021100005,1610612744,1610612746,-19.0


In [17]:
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['Team_ID','Game_ID'],how='inner')
nba_df = nba_df.merge(df_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['Team_ID','Game_ID'],
                      suffixes=['_H','_A'],how='inner')
nba_df['GAME_DATE']=nba_df['GAME_DATE_H']

In [18]:
for col in [col for col in nba_df.columns if ("Team_" in col)|("Game_ID" in col)|("MATCHUP" in col)|("GAME_DATE_" in col)]:
    del nba_df[col]

In [19]:
gc.collect()

415

In [20]:
nba_df.GAME_DATE = nba_df.GAME_DATE.apply(lambda dt : datetime.strptime(dt, "%b %d, %Y"))

In [21]:
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_HOME','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],how='inner')
nba_df = nba_df.merge(plus_minus_,left_on=['TEAM_ID_AWAY','GAME_ID'],right_on=['TEAM_ID','GAME_ID'],
                      suffixes=['_H','_A'],how='inner')

In [22]:
nba_df.tail()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS,LAST_1_GAME_FGM_H,LAST_1_GAME_FGA_H,LAST_1_GAME_FG_PCT_H,LAST_1_GAME_FG3M_H,LAST_1_GAME_FG3A_H,LAST_1_GAME_FG3_PCT_H,...,LAST_1_GAME_PLUS_MINUS_A,LAST_2_GAME_PLUS_MINUS_A,LAST_3_GAME_PLUS_MINUS_A,LAST_4_GAME_PLUS_MINUS_A,LAST_5_GAME_PLUS_MINUS_A,LAST_6_GAME_PLUS_MINUS_A,LAST_7_GAME_PLUS_MINUS_A,LAST_8_GAME_PLUS_MINUS_A,LAST_9_GAME_PLUS_MINUS_A,LAST_10_GAME_PLUS_MINUS_A
13413,0021800586,1610612746,1610612753,10.0,43.0,84.0,0.512,10.0,24.0,0.417,...,-17.0,28.0,-25.0,-10.0,-25.0,15.0,14.0,-3.0,-6.0,4.0
13414,0021800587,1610612737,1610612748,24.0,37.0,85.0,0.435,14.0,41.0,0.341,...,25.0,24.0,6.0,3.0,-27.0,-3.0,23.0,17.0,-10.0,7.0
13415,0021800588,1610612760,1610612764,-18.0,40.0,94.0,0.426,8.0,25.0,0.320,...,-6.0,-11.0,-16.0,-18.0,-8.0,-7.0,-7.0,-8.0,-15.0,14.0
13416,0021800589,1610612761,1610612754,16.0,42.0,82.0,0.512,14.0,31.0,0.452,...,3.0,8.0,8.0,-3.0,12.0,22.0,-1.0,-8.0,5.0,33.0
13417,0021800590,1610612756,1610612766,-6.0,41.0,77.0,0.532,9.0,19.0,0.474,...,-13.0,-4.0,-2.0,-2.0,-16.0,12.0,-17.0,-1.0,-5.0,-24.0


# Player Data Combine

In [23]:
playerBoxScoreAdv = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_2012to2019.pkl")

In [24]:
playerBoxScoreAdv = playerBoxScoreAdv[~playerBoxScoreAdv.MIN.isnull()]
playerBoxScoreAdv['MIN'] = playerBoxScoreAdv.MIN.apply(lambda x : float(x.split(':')[0]) + float(x.split(':')[1])/60.0)

In [25]:
playerBoxScoreAdv = playerBoxScoreAdv.loc[:,['GAME_ID','PLAYER_ID','TEAM_ID','MIN', 'OFF_RATING','DEF_RATING', 
                                             'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO','OREB_PCT', 'DREB_PCT',
                                             'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT','USG_PCT', 'PACE', 'PIE']]

In [26]:
playerBoxScoreAdv = playerBoxScoreAdv.fillna(0)

In [27]:
playerBoxScoreAdv["TEAM_ID"] = playerBoxScoreAdv["TEAM_ID"].astype(str)
playerBoxScoreAdv["PLAYER_ID"] = playerBoxScoreAdv["PLAYER_ID"].astype(str)

In [28]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [29]:
player_id = playerBoxScoreAdv.PLAYER_ID.unique()

In [30]:
playerBoxScoreAdv.index = [playerBoxScoreAdv.PLAYER_ID.values,playerBoxScoreAdv.GAME_ID.values]
playerBoxScoreAdv = playerBoxScoreAdv.sort_index()

In [31]:
player_id

array(['708', '2562', '979', ..., '1628518', '1629122', '1629022'],
      dtype=object)

In [32]:
# Raw Data
lastNgames = 10 
df_ = pd.DataFrame()
for player in player_id:
    player_i = playerBoxScoreAdv.loc[player]
    df_player = pd.DataFrame()
    
    for time in range(1,lastNgames+1):            
        df_player = pd.concat([df_player,
                               pd.DataFrame(player_i.shift(time).loc[:,"MIN":].values,
                                            columns=["LAST_"+str(time)+"_GAME_PLAYER_"+col for col in player_i.loc[:,"MIN":].columns])
                              ],axis=1)
        
    df_player = pd.concat([player_i.loc[:,"GAME_ID":"TEAM_ID"].reset_index(drop=True),df_player],axis=1)
    df_ = pd.concat([df_, df_player],axis=0).reset_index(drop=True)

In [33]:
df_.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_RAW_2012to2019.pkl")

# Add Player Data to Team

In [34]:
# raw data
df_ = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/BOXSCOREADVANCE_PLAYER_LAST_10_GAEMS_RAW_2012to2019.pkl")

In [35]:
playerBoxScoreAdv = playerBoxScoreAdv.reset_index(drop=True)

In [36]:
playerBoxScoreAdv = playerBoxScoreAdv.sort_values(["PLAYER_ID","GAME_ID"]).reset_index(drop=True)

In [37]:
game_shift = playerBoxScoreAdv.groupby(["PLAYER_ID"])["GAME_ID"].shift().to_frame("GAME_ID_LAST")

In [38]:
game_shift = pd.concat([game_shift,playerBoxScoreAdv[["GAME_ID","PLAYER_ID"]]],axis=1)

In [39]:
df_["PLAYER_ID"] = df_.PLAYER_ID.astype(str)

In [40]:
game_shift = game_shift.merge(df_, left_on=["GAME_ID_LAST","PLAYER_ID"], right_on=["GAME_ID","PLAYER_ID"],suffixes=["","_LAST"], how="left")

In [41]:
game_shift.drop("GAME_ID_LAST",axis=1,inplace=True)

In [42]:
game_shift = game_shift[~game_shift.TEAM_ID.isna()].reset_index(drop=True)

In [43]:
game_shift = game_shift.fillna(0)

In [44]:
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(int)
game_shift["TEAM_ID"] = game_shift.TEAM_ID.astype(str)

In [45]:
game_shift = game_shift.sort_values(["GAME_ID","TEAM_ID","PLAYER_ID"]).reset_index(drop=True)

In [46]:
game_shift["PLAYER_MIN_RATIO"] = game_shift.groupby(["GAME_ID","TEAM_ID"]).apply(lambda x : x["LAST_1_GAME_PLAYER_MIN"]/sum(x["LAST_1_GAME_PLAYER_MIN"])).values

In [47]:
game_shift = game_shift[~game_shift.PLAYER_MIN_RATIO.isna()].reset_index(drop=True)

In [48]:
game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":] = game_shift.loc[:,"LAST_1_GAME_PLAYER_MIN":].apply(lambda x : (x * x["LAST_1_GAME_PLAYER_MIN"])/240.0,axis=1)

In [49]:
game_shift = game_shift.groupby(["GAME_ID","TEAM_ID"]).sum().reset_index()

In [50]:
game_shift.head()

,GAME_ID,TEAM_ID,LAST_1_GAME_PLAYER_MIN,LAST_1_GAME_PLAYER_OFF_RATING,LAST_1_GAME_PLAYER_DEF_RATING,LAST_1_GAME_PLAYER_NET_RATING,LAST_1_GAME_PLAYER_AST_PCT,LAST_1_GAME_PLAYER_AST_TOV,LAST_1_GAME_PLAYER_AST_RATIO,LAST_1_GAME_PLAYER_OREB_PCT,...,LAST_10_GAME_PLAYER_OREB_PCT,LAST_10_GAME_PLAYER_DREB_PCT,LAST_10_GAME_PLAYER_REB_PCT,LAST_10_GAME_PLAYER_TM_TOV_PCT,LAST_10_GAME_PLAYER_EFG_PCT,LAST_10_GAME_PLAYER_TS_PCT,LAST_10_GAME_PLAYER_USG_PCT,LAST_10_GAME_PLAYER_PACE,LAST_10_GAME_PLAYER_PIE,PLAYER_MIN_RATIO
0,0021100022,1610612747,28.808736,95.758729,99.478361,-3.729979,0.165185,1.128056,18.783562,0.035987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.120036
1,0021100024,1610612748,26.501701,104.601938,87.774542,16.826146,0.123284,0.772721,12.838507,0.039480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114627
2,0021100027,1610612760,26.982500,103.898368,92.068965,11.851618,0.176137,0.697593,14.458326,0.022754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112427
3,0021100028,1610612738,28.657603,101.719889,103.282500,-1.573576,0.181557,0.790389,22.506146,0.009162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126850
4,0021100032,1610612744,24.392160,77.093799,95.293104,-18.199306,0.099826,0.468785,7.394035,0.027391,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118870


# Combine Player and Team infomation

In [51]:
nba_df = nba_df[~nba_df.LAST_10_GAME_PLUS_MINUS_A.isna()].reset_index(drop=True)

In [52]:
nba_df.head()

,GAME_ID,TEAM_ID_HOME,TEAM_ID_AWAY,PLUS_MINUS,LAST_1_GAME_FGM_H,LAST_1_GAME_FGA_H,LAST_1_GAME_FG_PCT_H,LAST_1_GAME_FG3M_H,LAST_1_GAME_FG3A_H,LAST_1_GAME_FG3_PCT_H,...,LAST_1_GAME_PLUS_MINUS_A,LAST_2_GAME_PLUS_MINUS_A,LAST_3_GAME_PLUS_MINUS_A,LAST_4_GAME_PLUS_MINUS_A,LAST_5_GAME_PLUS_MINUS_A,LAST_6_GAME_PLUS_MINUS_A,LAST_7_GAME_PLUS_MINUS_A,LAST_8_GAME_PLUS_MINUS_A,LAST_9_GAME_PLUS_MINUS_A,LAST_10_GAME_PLUS_MINUS_A
0,0021100222,1610612739,1610612741,-39.0,35.0,82.0,0.427,5.0,15.0,0.333,...,-16.0,9.0,11.0,-15.0,14.0,16.0,13.0,10.0,-8.0,1.0
1,0021100244,1610612741,1610612751,15.0,40.0,85.0,0.471,6.0,15.0,0.400,...,-10.0,-13.0,7.0,-8.0,12.0,-19.0,-16.0,-7.0,-16.0,6.0
2,0021100250,1610612757,1610612758,12.0,34.0,81.0,0.420,3.0,20.0,0.150,...,-33.0,2.0,-13.0,-39.0,-14.0,7.0,-27.0,-27.0,-17.0,-22.0
3,0021100254,1610612748,1610612739,7.0,25.0,67.0,0.373,3.0,12.0,0.250,...,-27.0,8.0,-5.0,11.0,-8.0,-20.0,11.0,-15.0,-7.0,16.0
4,0021100255,1610612756,1610612761,-3.0,31.0,83.0,0.373,6.0,20.0,0.300,...,-12.0,-23.0,-9.0,-13.0,-15.0,-35.0,5.0,-6.0,-13.0,8.0


In [53]:
game_shift.head()

,GAME_ID,TEAM_ID,LAST_1_GAME_PLAYER_MIN,LAST_1_GAME_PLAYER_OFF_RATING,LAST_1_GAME_PLAYER_DEF_RATING,LAST_1_GAME_PLAYER_NET_RATING,LAST_1_GAME_PLAYER_AST_PCT,LAST_1_GAME_PLAYER_AST_TOV,LAST_1_GAME_PLAYER_AST_RATIO,LAST_1_GAME_PLAYER_OREB_PCT,...,LAST_10_GAME_PLAYER_OREB_PCT,LAST_10_GAME_PLAYER_DREB_PCT,LAST_10_GAME_PLAYER_REB_PCT,LAST_10_GAME_PLAYER_TM_TOV_PCT,LAST_10_GAME_PLAYER_EFG_PCT,LAST_10_GAME_PLAYER_TS_PCT,LAST_10_GAME_PLAYER_USG_PCT,LAST_10_GAME_PLAYER_PACE,LAST_10_GAME_PLAYER_PIE,PLAYER_MIN_RATIO
0,0021100022,1610612747,28.808736,95.758729,99.478361,-3.729979,0.165185,1.128056,18.783562,0.035987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.120036
1,0021100024,1610612748,26.501701,104.601938,87.774542,16.826146,0.123284,0.772721,12.838507,0.039480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114627
2,0021100027,1610612760,26.982500,103.898368,92.068965,11.851618,0.176137,0.697593,14.458326,0.022754,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112427
3,0021100028,1610612738,28.657603,101.719889,103.282500,-1.573576,0.181557,0.790389,22.506146,0.009162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126850
4,0021100032,1610612744,24.392160,77.093799,95.293104,-18.199306,0.099826,0.468785,7.394035,0.027391,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118870


In [54]:
nba_df["TEAM_ID_HOME"] = nba_df["TEAM_ID_HOME"].astype(str)
nba_df["TEAM_ID_AWAY"] = nba_df["TEAM_ID_AWAY"].astype(str)
game_shift["TEAM_ID"] = game_shift["TEAM_ID"].astype(str)

In [55]:
game_shift = (nba_df[["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"]].merge(game_shift,left_on=["GAME_ID","TEAM_ID_HOME"],right_on=["GAME_ID","TEAM_ID"],how="left")
              .merge(game_shift,left_on=["GAME_ID","TEAM_ID_AWAY"],right_on=["GAME_ID","TEAM_ID"],how="left",suffixes=['_H','_A']))

In [56]:
game_shift.drop(["TEAM_ID_H","TEAM_ID_A","PLAYER_MIN_RATIO_A","PLAYER_MIN_RATIO_H"],axis=1,inplace=True)

In [57]:
gc.collect()

84

In [58]:
nba_df = nba_df.merge(game_shift, on=["GAME_ID", "TEAM_ID_HOME", "TEAM_ID_AWAY"],how="inner" )

In [59]:
nba_df.drop(["TEAM_ID_H","TEAM_ID_A"],axis=1,inplace=True)

In [60]:
nba_df.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_RAW_2012TO2019.pkl")

# Lottery Data

In [66]:
lottery_df = pd.read_pickle("LOTTERY_2012to2019.pkl")

In [ ]:
team = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_HOME_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True)
team = team.merge(pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/LEAGUEFINDER_TEAM_AWAY_2012to2019.pkl")[['GAME_ID', 'TEAM_ID', 'GAME_DATE']].reset_index(drop=True),
                 on=["GAME_ID","GAME_DATE"],how="inner",suffixes=["_HOME","_AWAY"])

In [ ]:
team["GAME_DATE"] = team.GAME_DATE.apply(lambda dt : datetime.strptime(dt,"%Y-%m-%d").date())

In [ ]:
lottery_with_gameid = lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")

In [ ]:
lottery_df["GAME_DATE"] = lottery_df.GAME_DATE.apply(lambda dt : dt + timedelta(days=-1))
lottery_with_gameid = pd.concat([lottery_with_gameid,lottery_df.merge(team, on=["TEAM_ID_HOME","TEAM_ID_AWAY","GAME_DATE"], how="inner")],
                                axis=0).reset_index(drop=True)

In [ ]:
lottery_with_gameid = lottery_with_gameid.loc[:,[col for col in lottery_with_gameid.columns if col not in 
                                                 ['AWAY','HOME','TEAM_CH_NAME_HOME', 'TEAM_EN_NAME_HOME', 
                                                  'TEAM_CH_NAME_AWAY', 'TEAM_EN_NAME_AWAY','GAME_DATE']]]

In [ ]:
lottery_with_gameid.to_pickle("GAMEID_LOTTERY_2012to2019.pkl")

In [ ]:
lottery_df